In [17]:
import os
import json
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
import re
from transformers import BertModel, BertTokenizer
import torch

/Users/cansakiroglu/miniconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mode = 'read' # 'write' or 'read'
if mode == 'write':
    # To write the data
    path_to_data = 'data/'
    file_names = os.listdir(path_to_data)
    print('Number of files: ', len(file_names))
    data = []
    for i in tqdm(range(len(file_names))):
        with open(path_to_data + file_names[i], 'r') as f:
            data.append(BeautifulSoup(json.loads(f.read())['data'], 'html.parser').get_text())
    with open('data.pkl', 'wb') as f:
        pickle.dump(data, f)
else:
    # To read the data
    with open('data.pkl', 'rb') as f:
        data = pickle.load(f)

In [3]:
data[0]

' T.C.İSTANBUL5. ASLİYE TİCARET MAHKEMESİ ESAS NO\t: 2016/451 EsasKARAR NO\t: 2019/85DAVA\t: Alacak (Ticari Nitelikteki Hizmet Sözleşmesinden Kaynaklanan)DAVA TARİHİ\t: 21/04/2016KARAR TARİHİ\t: 05/02/2019Mahkememizde görülmekte olan Alacak davasının yapılan açık yargılaması sonunda,GEREĞİ DÜŞÜNÜLDÜ:Davacı vekili 21/04/2016 tarihli dava dilekçesinde; Taraflar ve tarafların dışındaki kişilerle yapılmış 20 Mart 2006 tarihli düzenleme şeklinde ortaklar özel sözleşmesi gereğince davalı ve dava dışı diğer şahısların taşınmazlarının kurulacak bir şirkete devredilmesi ve kurulmuş şirketin bu taşınmazlar üzerinde inşaat faaliyetine başlamasını öngören bir sözleşme yapıldığını, bu sözleşmeye göre aralarında davalının da bulunduğu "mal sahiplerine" ait taşınmazların üzerinde yapılacak inşaatın finansmanının davacı tarafından karşılanacağını, taşınmaz üzerindeki borçların ve bazı giderlerin davacı tarafından ödendiğini, yapılacak inşaata ilişkin, öngörülen biçimde ruhsat alınmasının yasal olarak 

In [5]:
queries = []
for i in tqdm(range(len(data))):
    pattern = r"[Dd]ava dilekçesinde.*?;.*?[qwertyuıopğüasdfghjklşizxcvbnmöç]\.[ ]?[QWERTYUIOPĞÜASDFGHJKLŞİZXCVBNMÖÇ]"
    match = re.search(pattern, data[i])
    result = match.group(0) if match else None
    queries.append(result.split(';')[1].strip() if result is not None else None)
    if match:
        data[i] = re.sub(pattern, result[-1], data[i])


100%|██████████| 446094/446094 [00:53<00:00, 8367.48it/s] 


In [6]:
queries[0]

In [7]:
data[0]

' T.C.İSTANBUL5. ASLİYE TİCARET MAHKEMESİ ESAS NO\t: 2016/451 EsasKARAR NO\t: 2019/85DAVA\t: Alacak (Ticari Nitelikteki Hizmet Sözleşmesinden Kaynaklanan)DAVA TARİHİ\t: 21/04/2016KARAR TARİHİ\t: 05/02/2019Mahkememizde görülmekte olan Alacak davasının yapılan açık yargılaması sonunda,GEREĞİ DÜŞÜNÜLDÜ:Davacı vekili 21/04/2016 tarihli Ltd. Şti.\'ne yapılan toplamda 32.450,00 TL ödemeden davalı hissesine düşenin 8.112,50 TL olduğunu, taşınmazlara ilişkin proje danışmanlık hizmet için ... Tic. A.Ş.\'ne 108.000,00 TL ödemeden davalı hissesine düşenin 27.000,00 TL olduğunu, İstanbul Kapatılan 48. A.T.M.\'nin 2013/101 esas, 2014/72 karar sayılı mahkeme ilamı, 3.sayfa son paragrafta "taşınmazların devir tarihinde ödenecek olan miktarın belirlendiğini, bu miktarın 2.300,000 Amerikan Dolarının davacılara (mal sahiplerine) ödendiğini, davacılar tarafından sunulan 14.05.2008 havale tarihli replik dilekçesinde kabul edilmiştir." denildiğini, davacı tarafından bu şahsın borçlarının ödenmesinden doğan

In [8]:
null_count = sum(1 for item in queries if item is None)
 
print(f"Number of null elements in queries: {null_count}")

Number of null elements in queries: 158635


In [9]:
total_count = len(queries)
non_null_count = total_count - null_count
ratio = round(non_null_count / total_count, 4)
print(f"Successful pair rate: {ratio*100}%")

Successful pair rate: 64.44%


In [10]:
dataset_dict = {'doc': [], 'query': []}
for i in tqdm(range(len(data))):
    if queries[i] is not None and data[i] is not None:
        dataset_dict['doc'].append(data[i])
        dataset_dict['query'].append(queries[i])

  0%|          | 0/446094 [00:00<?, ?it/s]

100%|██████████| 446094/446094 [00:30<00:00, 14539.10it/s]


In [11]:
dataset = pd.DataFrame(dataset_dict)

In [12]:
dataset

,doc,query
0,T.C. İstanbul Anadolu 8. ASLİYE TİCARET MAHK...,"müvekkilinin ortak ve yetkilisi olduğu, ---- ..."
1,T.C.İSTANBUL5. ASLİYE TİCARET MAHKEMESİESAS N...,müvekkiline ait ... ENERJİ SAN. TİC. LTD. ŞTİ....
2,T.C. BAKIRKÖY 7. ASLİYE TİCARET MAHKEMESİ\t E...,"27.08.2012 tarihli, ...... numaralı Poliçe ile..."
3,T.C. BAKIRKÖY 4. ASLİYE TİCARET MAHKEMESİ\t E...,...Bankası A.Ş. ... Şubesine ait
4,T.C.İSTANBUL14. ASLİYE TİCARET MAHKEMESİESAS ...,"""davalı borçlu ile müvekkili arasında 14/04/20..."
...,...,...
287454,....TÜRK MİLLETİ ADINA\tT.C.\tBURSA1. ASLİYE...,davacı ve davalı arasında olan ticari ilişki n...
287455,T.C. İstanbul Anadolu 1. ASLİYE TİCARET MAHK...,davacının belirtilen tarihler arasında çalıştı...
287456,T.C. İstanbul Anadolu 3. ASLİYE TİCARET MAHK...,Müvekkili ----- başlangıç tarihli---- poliçesi...
287457,T.C.İSTANBULBÖLGE ADLİYE MAHKEMESİ14. HUKUK D...,müvekkilinin davalıya satmış olduğu mal ve hiz...


In [16]:
# Load pre-trained Turkish BERT model and tokenizer
model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

NameError: name 'BertTokenizer' is not defined

In [ ]:
# Tokenize and convert to tensor
inputs = tokenizer(dataset['query'].tolist(), return_tensors="pt", truncation=True, padding=True)

# Forward pass through the BERT model
with torch.no_grad():
    outputs = model(**inputs)

# Extract the embeddings from the output
embeddings = outputs.last_hidden_state.mean(dim=1)

# Convert to NumPy array
embeddings_np = embeddings.numpy()

In [ ]:
# Tokenize and convert to tensor
inputs = tokenizer(dataset['doc'].tolist(), return_tensors="pt", truncation=True, padding=True)

# Forward pass through the BERT model
with torch.no_grad():
    outputs = model(**inputs)

# Extract the embeddings from the output
embeddings = outputs.last_hidden_state.mean(dim=1)

# Convert to NumPy array
embeddings_np = embeddings.numpy()